<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Retreive pages inside a Database
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Notion/Notion_Get_database.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

## Input

### Import library

In [1]:
from notion import Notion
import naas

### Variables

In [2]:
# Enter Notion Token API
token = "secret_R1CrUGn8bx9itbJW0Fc9Cc0R9Lmhbnz2ayqEe0GhRPq"

# Enter database url
url = "https://www.notion.so/d0bb915c4cb4422a87679f3bb9658282?v=8cd7c6ea0ec244da9eb65aa88a87aabb"

In [3]:
notion = Notion().connect(token)

## Model

### Get all pages

In [4]:
df = notion.database.get(url)

- [ ] Add the Id to let user retreive a page inside the database
### Display result

In [5]:
df

,DATE,STATUS,JOBS,COMPLETION TIME,EMAIL,MARIED,INTERVIEWER,TASK,PHONE NUMBER,NAME
0,2020-08-20,Cancelled,Rêveur 🚀,4,axelito@gmail.com,False,,Onboard Flow,000,Axel
1,2021-08-11,Scheduled,dancer,3,rasseax@gmail.com,False,Axel Rasse,Onboarding Flow,None,Tom Simon
2,2019-10-24T14:00:00.000-07:00,Scheduled,surfer,25,chelsea.zhang@gmail.com,False,,Onboarding Flow,None,Chelsea Zhang
